### Create an API session

In [13]:
import pandas as pd

from weaveapi.records import *
from weaveapi.options import *
from weaveapi.filter import *

from weaveh import *

WEAVE_CONFIG = "config/demo_client_ethsf.config"
nodeApi, session = connect_weave_api(WEAVE_CONFIG)

{'organization': 'weavedemo', 'account': 'ioan', 'scope': '*', 'chainClientConfig': {'apiVersion': 1, 'seed': '1a2bac41a628fb7b54ca84bd3e1a2bac', 'privateKeyFile': '/app/config/node2.pvk', 'publicKeyFile': '/app/config/node2.pub', 'http': {'host': 'ethsf22.weavechain.com', 'port': 443, 'useHttps': True}}}
{"res":"ok","data":"pong 1667307920813"}


### Get directory table layout

In [59]:
scope = "ethsf22"
table = "directory"

reply = nodeApi.getTableDefinition(session, scope, table).get()
#print(reply)
layout = json.loads(reply["data"])["layout"]
layout["columns"]

[{'columnName': 'id',
  'type': 'LONG',
  'readTransform': 'NONE',
  'isIndexed': True,
  'isNullable': False,
  'isUnique': False,
  'isEncrypted': False,
  'allowPlaintext': True},
 {'columnName': 'name_nickname',
  'type': 'STRING',
  'readTransform': 'NONE',
  'isIndexed': False,
  'isNullable': True,
  'isUnique': False,
  'isEncrypted': False,
  'allowPlaintext': True},
 {'columnName': 'name_last',
  'type': 'STRING',
  'readTransform': 'NONE',
  'isIndexed': False,
  'isNullable': True,
  'isUnique': False,
  'isEncrypted': False,
  'allowPlaintext': True},
 {'columnName': 'name_first',
  'type': 'STRING',
  'readTransform': 'NONE',
  'isIndexed': False,
  'isNullable': True,
  'isUnique': False,
  'isEncrypted': False,
  'allowPlaintext': True},
 {'columnName': 'email',
  'type': 'STRING',
  'readTransform': 'NONE',
  'isIndexed': False,
  'isNullable': True,
  'isUnique': False,
  'isEncrypted': False,
  'allowPlaintext': True},
 {'columnName': 'phone',
  'type': 'STRING',
  '

### Mark few columns for erasure on Read

In [60]:
newLayout = layout.copy()
del newLayout["layout"]
del newLayout["indexes"]
del newLayout["columnNames"]
newLayout["columns"] = { i["columnName"]: i for i in layout["columns"]}

newLayout["columns"]["phone"]["readTransform"] = "ERASURE"
newLayout["columns"]["address_city"]["readTransform"] = "ERASURE"
newLayout["columns"]["birthday"]["readTransform"] = "ERASURE"

reply = nodeApi.updateLayout(session, scope, table, newLayout).get()
print(reply)


{'res': 'ok', 'target': {'operationType': 'UPDATE_LAYOUT', 'organization': 'weavedemo', 'account': 'ioan', 'scope': 'ethsf22', 'table': 'directory'}}


### Check birthday has erasure

In [64]:
reply = nodeApi.getTableDefinition(session, scope, table).get()
#print(reply)
layout = json.loads(reply["data"])["layout"]
layout["columns"][-1]

{'columnName': 'birthday',
 'type': 'STRING',
 'readTransform': 'ERASURE',
 'isIndexed': False,
 'isNullable': True,
 'isUnique': False,
 'isEncrypted': False,
 'allowPlaintext': True}

### Read records

In [69]:
reply = nodeApi.read(session, scope, table, None, READ_DEFAULT_NO_CHAIN).get()
#raw = nodeApi.download(session, scope, table, None, "csv", READ_DEFAULT).get()
#print(reply)
df = pd.DataFrame(reply["data"])

df.head()

,id,name_nickname,name_last,name_first,email,phone,addres_country,address_city,linkedin_url,discord_username,telegram_username,eth_wallet,birthday
0,1,nickname,lastname,firstname,email@email.com,,USA,,https://linkedin.com/in/test,name#1234,@test,0x1234,


### Compute average age with (degenerated 1:1) MPC

In [80]:
import time

nodes = ALL_ACTIVE_NODES
#nodes = [ "weavejjf8yGD2Gez24EtKPVFSmX7ZmR4itmF4BU5p1RLqt6Vn", "weavexy5oNNScffcJBPTxeTCc4dE5XZwSYEMh9XsqLSMNTPVq" ]

algo = "mean"

reply = nodeApi.mpc(session, scope, table, algo, [ "birthday" ], None, MPCOptions(False, 30, nodes)).get()

#returned value is dates average in unix timestamp, transforming it to (approximate) years
avg = round(time.time() * 1000) - float(reply["data"])
days = avg / (24 * 3600 * 1000)

print("Average age:", days / 365.25)

Average age: 32.83393707138693
